## 참고: https://hleecaster.com/ml-linear-regression-example/

## 라이브러리 설치, 호출

In [ ]:
# pip install tensorflow

In [ ]:
# pip install statsmodels

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau

## 데이터 다운로드 (특별할인 판매)

In [2]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/jmnote/zdata/master/logistic-regression/special-sales.csv')
print(df.head())
print(df.shape)

         date weekday  busy_day  high_temperature  special_sales
0  2002-08-05     Mon         0                28              1
1  2002-08-06     Tue         0                24              0
2  2002-08-07     Wed         1                26              0
3  2002-08-08     Thu         0                24              0
4  2002-08-09     Fri         0                23              0
(21, 5)


## Input, Feature 설정

In [3]:
Label = df['special_sales']
InputFeature = df[['busy_day','high_temperature']]

## Keras Logit 모델 fitting

In [10]:
model = Sequential()
model.add(Dense(3, activation='linear', input_shape=(2,)))
model.add(Dense(3, activation='linear'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])

## Keras 모델 살펴보기

In [11]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 3)                 9         
                                                                 
 dense_6 (Dense)             (None, 3)                 12        
                                                                 
 dense_7 (Dense)             (None, 1)                 4         
                                                                 
Total params: 25
Trainable params: 25
Non-trainable params: 0
_________________________________________________________________


## Call-back 함수
## 모델 학습

In [12]:
# Call-back 함수
# CheckPoint: Epoch 마다 validation 성능을 검증하여, best performance 일 경우 저장
CP = ModelCheckpoint(filepath='-{epoch:03d}-{loss:.4f}-{accuracy:.4f}.hdf5',
            monitor='loss', verbose=1, save_best_only=True, mode='min')

# Learning Rate 줄여나가기
LR = ReduceLROnPlateau(monitor='loss',factor=0.8,patience=3, verbose=1, min_lr=1e-8)

CALLBACK = [CP, LR]

In [13]:
model.fit(x=InputFeature, y=Label, epochs=100, shuffle=True, batch_size=3, callbacks=CALLBACK)

Epoch 1/100
1/7 [===>..........................] - ETA: 6s - loss: 4.9460 - accuracy: 0.3333
Epoch 1: loss improved from inf to 2.38542, saving model to -001-2.3854-0.4286.hdf5
7/7 [==============================] - 1s 10ms/step - loss: 2.3854 - accuracy: 0.4286 - lr: 0.0100
Epoch 2/100
1/7 [===>..........................] - ETA: 0s - loss: 0.7319 - accuracy: 0.6667
Epoch 2: loss improved from 2.38542 to 1.42925, saving model to -002-1.4293-0.6190.hdf5
7/7 [==============================] - 0s 12ms/step - loss: 1.4293 - accuracy: 0.6190 - lr: 0.0100
Epoch 3/100
1/7 [===>..........................] - ETA: 0s - loss: 0.0623 - accuracy: 1.0000
Epoch 3: loss improved from 1.42925 to 0.76408, saving model to -003-0.7641-0.6190.hdf5
7/7 [==============================] - 0s 10ms/step - loss: 0.7641 - accuracy: 0.6190 - lr: 0.0100
Epoch 4/100
1/7 [===>..........................] - ETA: 0s - loss: 0.7355 - accuracy: 0.3333
Epoch 4: loss did not improve from 0.76408
7/7 [=======================

# Model Load 하기 전, hdf5 파일 이름 꼭 확인하기

In [15]:
# epoch - 99번째, loss - 0.5984, accuracy - 0.7619 
model.load_weights("-098-0.5455-0.7619.hdf5")

- model의 dense를 늘렸을 경우, 바뀐 model file로 load 해야 함 <br>
( 당연한거지만 이 전에 학습한 것이 값이 더 괜찮아서 load 하려고 했다가 Layer mismatch error 발생)

## FLASK 셋팅하기

In [16]:
from flask import Flask
from flask import render_template

In [17]:
app = Flask(__name__)

# FLASK API 구현부분

In [18]:
@app.route('/SpecialSale/<busy_day>/<high_temperature>')
def PredictionSpecialSale(busy_day=None, high_temperature=None):
    Input = pd.DataFrame({
        'busy_day':[int(busy_day)],
        'high_temperature':[float(high_temperature)]
    })
    
    return str(model.predict(Input)[0][0])

# Flask, port 5000으로 실행

In [19]:
app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.5:5000/ (Press CTRL+C to quit)


1/1 [==============================] - 0s 246ms/step


192.168.0.5 - - [07/Jul/2022 09:46:15] "GET /SpecialSale/1/38 HTTP/1.1" 200 -
192.168.0.5 - - [07/Jul/2022 09:46:15] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 50ms/step


...

192.168.0.5 - - [07/Jul/2022 09:50:28] "GET /SpecialSale/13/38 HTTP/1.1" 200 -


1/1 [==============================] - 0s 46ms/step


192.168.0.5 - - [07/Jul/2022 09:50:41] "GET /SpecialSale/12/38 HTTP/1.1" 200 -


1/1 [==============================] - 0s 44ms/step


192.168.0.5 - - [07/Jul/2022 09:51:10] "GET /SpecialSale/90/38 HTTP/1.1" 200 -


- http://172.30.1.6:5000/SpecialSale/1/38 => 0.5882789
- http://172.30.1.6:5000/SpecialSale/2/38 => 0.87821376
- http://172.30.1.6:5000/SpecialSale/3/38 => 0.97325736
- http://172.30.1.6:5000/SpecialSale/4/38 => 0.99458504
- 계속 증가하다가 12를 기점으로 1.0 유지